# Importer les données du fichier "variables"


In [2]:
import numpy as np
import pandas as pd
import pickle

x=np.load("variables/x.npy")
y=np.load("variables/y.npy")
x_train=np.load("variables/x_train.npy")
x_test=np.load("variables/x_test.npy")
y_test=np.load("variables/y_test.npy")
y_train=np.load("variables/y_train.npy")
y_train=np.load("variables/y_train.npy")
x_1 = pd.read_pickle("variables/x_1")
x_train_set = pd.read_pickle("variables/x_train_set")
x_test_set = pd.read_pickle("variables/x_test_set")
main_data = pd.read_pickle("variables/main_data")
best_mlp = pickle.load(open('variables/mlp_model.sav', 'rb')) 
best_RF_clf = pickle.load(open('variables/best_RF_clf.sav', 'rb'))

# Calculer les caractéristiques  les plus infulantes


In [3]:
feature_importances = best_RF_clf.feature_importances_

# Convertire en critères
 En out put on retrouver les critères du plus pertinants au moins ifluants

In [9]:
extra_attribs= [  "Department" , "Gender",  "JobRole", "MaritalStatus"]

main_data_num= main_data.select_dtypes(include=[np.number]) 
num_attribs = list(main_data_num)

attributes = num_attribs + extra_attribs 
best_arg = sorted(zip(feature_importances, attributes), reverse=True)
print("Ci dessous sont regroupés les critères du plus pertinants au moins influant :")
print("")
print(best_arg)

Ci dessous sont regroupés les critères du plus pertinants au moins influant :

[(0.09824789518229983, 'WorkMeanTime'), (0.07382028553606194, 'Age'), (0.06952828962653161, 'MonthlyIncome'), (0.06549642339848573, 'TotalWorkingYears'), (0.058244520218226126, 'YearsAtCompany'), (0.05207455423267441, 'DistanceFromHome'), (0.05116387179955666, 'PercentSalaryHike'), (0.04464988135724109, 'YearsWithCurrManager'), (0.04325552655044438, 'NumCompaniesWorked'), (0.03708155147841288, 'YearsSinceLastPromotion'), (0.03637152706622121, 'JobSatisfaction'), (0.03375782023858783, 'EnvironmentSatisfaction'), (0.0329274134466722, 'TrainingTimesLastYear'), (0.03226630154240355, 'WorkLifeBalance'), (0.030493051653678827, 'JobLevel'), (0.029981177700106075, 'JobInvolvement'), (0.027444499254844976, 'StockOptionLevel'), (0.011642871052597542, 'JobRole'), (0.011392341390249222, 'MaritalStatus'), (0.010973083734865469, 'Gender'), (0.009542033767510442, 'PerformanceRating'), (0.007874471130970472, 'Department'), 

# On supprime les valeurs avec le poids le plus faible 

In [11]:
x_train_set.drop("EmployeeCount",axis =1 ,inplace = True)
x_train_set.drop("StandardHours",axis =1 ,inplace = True)
x_train_set.drop("Department",axis =1 ,inplace = True)

x_test_set.drop("EmployeeCount",axis =1 ,inplace = True)
x_test_set.drop("StandardHours",axis =1 ,inplace = True)
x_test_set.drop("Department",axis =1 ,inplace = True)

x_1.drop("EmployeeCount", axis=1, inplace=True)
x_1.drop("StandardHours", axis=1, inplace=True)
x_1.drop("Department", axis=1, inplace=True)


# On standarise les données pour l'entrainment du model

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

x_1_num = x_1.select_dtypes(include=[np.number]) 

num_attribs = list(x_1_num)
cat_attribs= [  "Gender",  "JobRole", "MaritalStatus"]
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
         ("cat", OneHotEncoder(), cat_attribs),
    ])


In [17]:

x_train = full_pipeline.fit_transform(x_train_set)
x_test = full_pipeline.fit_transform(x_test_set)

x = full_pipeline.fit_transform(x_1)

# On entraine le MLP le meilleur model pour notre etude

In [18]:
%%capture --no-display
best_mlp.fit(x_train, y_train)

MLPClassifier(activation='tanh', alpha=0.05, max_iter=300, random_state=1)

In [19]:
%%capture --no-display
from sklearn.model_selection import cross_val_score

best_mlp_scores = cross_val_score(best_mlp, x, y, cv=10, scoring="f1")


In [ ]:
print("Scores:", best_mlp_scores)
print("Mean:", best_mlp_scores.mean())
print("Standard deviation:", best_mlp_scores.std())
